In [72]:
from hmm.old_hmm import HMM, sample_poisson_stimuli
from hmm.hmm import HMM as HMM_new

import numpy as np

In [73]:
training_data = np.genfromtxt("../../data/Ex_2.csv", delimiter="," ,dtype=int)[1:, 1:]

In [74]:
gamma = 0.1
beta = 0.2
alpha = 0.5
rates = [1, 5]

# This is uppercase-gamma.
transition_matrix = np.array(
    [[1 - gamma, 0, gamma],
     [0, 1 - gamma, gamma],
     [beta / 2, beta / 2, 1 - beta]]
)

In [75]:
hmm = HMM(transition_matrix, alpha, lambda z: sample_poisson_stimuli(z, rates), states=[0, 1, 2], rates=rates)
hmm_new = HMM_new(transition_matrix, alpha, processing_modes=[0, 1, 2], rates=rates)

In [76]:
num_nodes = 8
time_steps = 1000
initial_c = 2

In [77]:
true_processing_modes, true_focus, observations = hmm.forward(
    num_nodes,
    time_steps,
    initial_c,
)

In [78]:
np.array(true_processing_modes)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1,
       1, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1, 2, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

Dimensions of the joint distribution will be (T - 1, num possible Cs at t, num possible Cs at t +1)

> ...it may be more convenient to test the implementation in other ways. You may, for instance, observe that
>
> $$P?(Z_{t,i}=z)-P(Z_{t,i}=z|X=x)$$
>
> has mean 0 and likewise for Ct. Using simulations you can compute such quantities, with $P(Z_{t,i}=z|X=x)$ computed by the inference algorithm, and empirically check if their averages across many replications of the simulations are zero.

In [79]:
num_simulations = 10
correct_C = 0
correct_Z = 0

diff_Z0 = []
diff_Z1 = []

for _ in range(num_simulations):
    true_processing_modes, true_focus, observations = hmm.forward(num_nodes, time_steps, initial_c)
    joint_probabilities_normalised = hmm.infer(observations)
    # Compute the marginal probabilities of C at each time step
    marginal_prob_C = np.sum(joint_probabilities_normalised, axis=2)

    # Calculate the estimated C at each time step
    estimated_C = np.argmax(marginal_prob_C, axis=1)

    # Compute the most likely Z given the estimated C
    estimated_Z = np.zeros((time_steps, num_nodes), dtype=int)

    for t, c in enumerate(estimated_C):
        estimated_Z[t] = hmm.sample_hidden_z(num_nodes, c)

    # Compute the accuracy
    correct_C += np.sum(true_processing_modes[:-1] == estimated_C) / time_steps
    correct_Z += np.sum(true_focus == estimated_Z) / (time_steps * num_nodes)

    # Calculate the marginal probability of Zt,i=0 given X=x
    marginal_prob_Z0_given_X = np.sum(joint_probabilities_normalised[:, :, :2], axis=2)
    # Calculate the marginal probability of Zt,i=1 given X=x
    marginal_prob_Z1_given_X = 1 - marginal_prob_Z0_given_X

    P_Z0 = np.mean(true_focus[:-1] == 0, axis=1)
    P_Z1 = np.mean(true_focus[:-1] == 1, axis=1)

    diff_Z0.append(P_Z0 - np.mean(marginal_prob_Z0_given_X, axis=1))
    diff_Z1.append(P_Z1 - np.mean(marginal_prob_Z1_given_X, axis=1))



In [80]:
correct_C /= num_simulations
correct_Z /= num_simulations

# Compute the mean differences across all simulations
mean_diff_Z0 = np.mean(np.array(diff_Z0))
mean_diff_Z1 = np.mean(np.array(diff_Z1))

print(f"Proportion of correct C estimations: {correct_C:.2f}")
print(f"Proportion of correct Z estimations: {correct_Z:.2f}")

print(f"Mean difference for Zt,i=0: {mean_diff_Z0:.5f}")
print(f"Mean difference for Zt,i=1: {mean_diff_Z1:.5f}")

Proportion of correct C estimations: 0.34
Proportion of correct Z estimations: 0.50
Mean difference for Zt,i=0: 0.27700
Mean difference for Zt,i=1: -0.27700


We then run the implementation on the real data

In [81]:
joint_probabilities_normalised = hmm.infer(observations)

# Compute the marginal probabilities of C at each time step
marginal_prob_C = np.sum(joint_probabilities_normalised, axis=2)

# Calculate the estimated C at each time step
estimated_C = np.argmax(marginal_prob_C, axis=1)

# Compute the most likely Z given the estimated C
estimated_Z = np.zeros((time_steps, num_nodes), dtype=int)

for t, c in enumerate(estimated_C):
    estimated_Z[t] = hmm.sample_hidden_z(num_nodes, c)

In [82]:
correct_C = np.sum(np.equal(estimated_C, true_processing_modes[:-1])) / time_steps
correct_Z = np.sum(estimated_Z == true_focus) / (time_steps * num_nodes)

print(f"Proportion of correct C estimations: {correct_C:.2f}")
print(f"Proportion of correct Z estimations: {correct_Z:.2f}")

Proportion of correct C estimations: 0.32
Proportion of correct Z estimations: 0.49


In [83]:
joint_probabilities_normalised = hmm.infer(training_data)
time_steps = joint_probabilities_normalised.shape[0]

In [84]:
joint_probabilities_normalised.shape

(99, 3, 3)

In [85]:
# Compute the marginal probabilities of C at each time step
marginal_prob_C = np.sum(joint_probabilities_normalised, axis=2)

# Calculate the estimated C at each time step
estimated_C = np.argmax(marginal_prob_C, axis=1)

# Compute the most likely Z given the estimated C
estimated_Z = np.zeros((time_steps, num_nodes), dtype=int)

for t, c in enumerate(estimated_C):
    estimated_Z[t] = hmm.sample_hidden_z(num_nodes, c)

In [86]:
estimated_C

array([0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0,
       0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0,
       0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0,
       0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2,
       2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0])

In [87]:
estimated_Z

array([[1, 0, 0, 0, 0, 1, 1, 0],
       [1, 0, 1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 1, 1],
       [0, 1, 0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [1, 1, 0, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 1, 1],
       [0, 0, 1, 1, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 1, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 1, 0],
       [1, 1, 1, 1, 0, 1, 0, 0],
       [1, 1, 0, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 1],
       [1, 1, 0, 1, 1, 1, 1, 0],
       [1, 1, 0, 0, 0, 0, 1, 0],
       [0, 1, 1, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 1, 1, 0],
       [1, 1, 0, 0, 0, 0, 1, 0],
       [1,

In [88]:
joint_probabilities_normalised

array([[[0.3       , 0.        , 0.03333333],
        [0.        , 0.3       , 0.03333333],
        [0.03333333, 0.03333333, 0.26666667]],

       [[0.3       , 0.        , 0.03333333],
        [0.        , 0.3       , 0.03333333],
        [0.03333333, 0.03333333, 0.26666667]],

       [[0.3       , 0.        , 0.03333333],
        [0.        , 0.3       , 0.03333333],
        [0.03333333, 0.03333333, 0.26666667]],

       [[0.3       , 0.        , 0.03333333],
        [0.        , 0.3       , 0.03333333],
        [0.03333333, 0.03333333, 0.26666667]],

       [[0.3       , 0.        , 0.03333333],
        [0.        , 0.3       , 0.03333333],
        [0.03333333, 0.03333333, 0.26666667]],

       [[0.3       , 0.        , 0.03333333],
        [0.        , 0.3       , 0.03333333],
        [0.03333333, 0.03333333, 0.26666667]],

       [[0.3       , 0.        , 0.03333333],
        [0.        , 0.3       , 0.03333333],
        [0.03333333, 0.03333333, 0.26666667]],

       [[0.3       ,